In [2]:
!pip -q install xgboost

In [52]:
from time import gmtime, strftime, sleep
from datetime import datetime
import re
from threading import Thread
import json
import random

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
import boto3
import sagemaker
from sagemaker import Session, image_uris
from sagemaker.serializers import CSVSerializer
from sagemaker.model import Model
from sagemaker.model_monitor import(DataCaptureConfig,
                                    ModelQualityMonitor,
                                    EndpointInput,
                                    CronExpressionGenerator)
from sagemaker.predictor import Predictor
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor.dataset_format import DatasetFormat

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

s3 = boto3.client("s3")
bucket = sess.default_bucket()
prefix = "churn-prediction-xgboost"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Setup S3 bucket
# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/datacapture
Ground truth path: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/ground_truth_data/2024-02-13-01-45-52
Report path: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/reports


Prepare dataset

In [5]:
df = pd.read_csv("data/Churn_Modelling.csv")
df.drop(columns=["RowNumber", "Surname"], inplace=True)
df = pd.get_dummies(df)
df.drop(columns="Gender_Male", inplace=True)
df

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female
0,15634602,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1
1,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1
2,15619304,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1
3,15701354,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1
4,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0
9996,15569892,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0
9997,15584532,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1
9998,15682355,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0


In [6]:
X = df.drop(columns="Exited")
y = df["Exited"]

X_train, X_test, y_train, y_test = train_test_split(
 X, y,
 test_size=0.2, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(
 X_test, y_test,
 test_size=0.5, random_state=1)

X_train.drop(columns="CustomerId", inplace=True)
X_val.drop(columns="CustomerId", inplace=True)

Upload to S3

In [7]:
train_file = "data/train_data.csv"
pd.concat([y_train, X_train], axis=1).to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "data/validation_data.csv"
pd.concat([y_val, X_val], axis=1).to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "data/batch_data.csv"
X_test.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/batch/batch_data.csv'

In [40]:
# save validation data with predictions without the CustomerId column
# this will be used for calculating a baseline for monitoring

pd.concat([y_test, X_test.iloc[:, 1:]],
          axis=1).to_csv("data/test_data_noID.csv",
                         index=False, header=False)

# save test data without predictions to simulate incoming data for the monitoring job
X_test.iloc[:, 1:].to_csv("data/test-dataset-input-cols.csv", index=False, header=False)

## Training job and model creation

Set values for hyperparameters  
Note that we got these values from the experimentation notebook

In [9]:
%%time

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=region, version="1.7-1" # latest version
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    colsample_bytree=0.7956926890881284,
    gamma=0.4224376554847273,
    eta=0.01698158072074776,
    max_depth=4,
    num_round=214,
    subsample=0.7127419099093599
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-02-13-01-45-52


2024-02-13 01:45:53 Starting - Starting the training job...
2024-02-13 01:46:08 Starting - Preparing the instances for training.........
2024-02-13 01:47:34 Downloading - Downloading input data...
2024-02-13 01:48:10 Downloading - Downloading the training image......
2024-02-13 01:49:15 Training - Training image download completed. Training in progress..[2024-02-13 01:49:27.025 ip-10-0-194-64.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-02-13 01:49:27.049 ip-10-0-194-64.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-02-13:01:49:27:INFO] Imported framework sagemaker_xgboost_container.training
[2024-02-13:01:49:27:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-02-13:01:49:27:INFO] No GPUs detected (normal if no gpus installed)
[2024-02-13:01:49:27:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-02-13:01:49:27:INFO] Determined 0 GPU(s) available on th

## Use batch transform on our test data. Since the ID column is present, we will update the output filter to keep only ID and prediction

Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'CustomerId') and the last column (the inference result i.e. the probability of customer churn)

In [10]:
sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

batch_file = "batch_data.csv"
# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)

sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]", # input_filter will filter out CustomerId
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-02-13-01-50-05-797
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-02-13-01-50-06-533


.........................................[2024-02-13:01:56:57:INFO] No GPUs detected (normal if no gpus installed)
[2024-02-13:01:56:58:INFO] No GPUs detected (normal if no gpus installed)
[2024-02-13:01:56:58:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-02-13 01:56:58 +0000] [19] [INFO] Sta

Let's display the output

In [11]:
def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,15731026,0.148549
1,15792565,0.132313
2,15710316,0.355720
3,15781347,0.071115
4,15694859,0.098018
5,15739194,0.132254
6,15723894,0.413681
7,15652527,0.212699


Create model entity

In [12]:
model_name = f"final-project-xgb-churn-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

image_uri = image
model_data = sm_estimator.model_data

model = Model(image_uri=image_uri, model_data=model_data, role=role, sagemaker_session=sess)

Deploy the model with data capture enabled

In [13]:
endpoint_name = f"final-project-xgboost-v1-wquality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = final-project-xgboost-v1-wquality-monitor-2024-02-13-0157
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-02-13-01-57-41-016
INFO:sagemaker:Creating endpoint-config with name final-project-xgboost-v1-wquality-monitor-2024-02-13-0157
INFO:sagemaker:Creating endpoint with name final-project-xgboost-v1-wquality-monitor-2024-02-13-0157


------!

Use predictor to invoke the model

In [15]:
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sess, serializer=CSVSerializer()
)

Generate a baseline for model quality performance

In [16]:
churn_cutoff = 0.5
validate_dataset = "validation_with_predictions.csv"

limit = 200  # Need at least 200 samples to compute standard deviations
i = 0
with open(f"data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # our header
    with open("data/test_data_noID.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > churn_cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

........................................................................................................................................................................................................
Done!


examine the file we just created

In [17]:
!head data/validation_with_predictions.csv

probability,prediction,label
0.1485494077205658,0,0
0.1323130875825882,0,0
0.35571980476379395,0,0
0.07111486047506332,0,0
0.09801846742630005,0,0
0.1322542130947113,0,0
0.4136806130409241,0,0
0.21269924938678741,0,0
0.08133785426616669,0,0


Upload the baseline data to s3

In [18]:
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/baselining/results


In [19]:
baseline_dataset_uri = S3Uploader.upload(f"data/{validate_dataset}", baseline_data_uri)
baseline_dataset_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/baselining/data/validation_with_predictions.csv'

#### Create a baselining job with validation dataset predictions

Define the model quality monitoring object and execute the model quality monitoring baseline job. Model monitor will automatically generate baseline statistics and constraints based on the validation dataset provided.

In [21]:
# Create the model quality monitoring object
churn_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sess,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [22]:
# Name of the model quality baseline job
baseline_job_name = f"final-project-xgb-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [23]:
# Execute the baseline suggestion job.
job = churn_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name final-project-xgb-baseline-job-2024-02-13-0217


.................................................................................!

explore the results of the baseline job

In [24]:
baseline_job = churn_model_quality_monitor.latest_baselining_job

binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,154
confusion_matrix.0.1,4
confusion_matrix.1.0,27
confusion_matrix.1.1,16
recall.value,0.372093
recall.standard_deviation,0.023344
precision.value,0.8
precision.standard_deviation,0.045735
accuracy.value,0.845771
accuracy.standard_deviation,0.010845


In [25]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.372093,LessThanThreshold
precision,0.8,LessThanThreshold
accuracy,0.845771,LessThanThreshold
true_positive_rate,0.372093,LessThanThreshold
true_negative_rate,0.974684,LessThanThreshold
false_positive_rate,0.025316,GreaterThanThreshold
false_negative_rate,0.627907,GreaterThanThreshold
auc,0.888137,LessThanThreshold
f0_5,0.650407,LessThanThreshold
f1,0.507937,LessThanThreshold


#### Generate prediction data for Model Quality  Monitoring

Start generating some artificial traffic.  The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [41]:
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = sess.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "data/test-dataset-input-cols.csv")
        except sess.sagemaker_runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever)
thread.start()

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [44]:
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show upsagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml

Found Capture Files:
s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/datacapture/final-project-xgboost-v1-wquality-monitor-2024-02-13-0157/AllTraffic/2024/02/13/02/52-09-001-4abf934b-c893-4c53-8c87-c1070ec37182.jsonl
 s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/datacapture/final-project-xgboost-v1-wquality-monitor-2024-02-13-0157/AllTraffic/2024/02/13/02/53-09-865-7e443c05-953a-464b-9724-505fa4b69158.jsonl
 s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/datacapture/final-project-xgboost-v1-wquality-monitor-2024-02-13-0

Let's observe one of the lines

In [46]:
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "534,52,1,0.0,3,1,1,104035.41,1,0,0,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.8094968795776367\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "f56e8349-801f-44df-82cc-2c88fd1a4317",
    "inferenceId": "131",
    "inferenceTime": "2024-02-13T02:54:10Z"
  },
  "eventVersion": "0"
}


#### 4.3 Generate synthetic ground truth

In [49]:
def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [51]:
NUM_GROUND_TRUTH_RECORDS = len(X_test)  # the number of rows in data we're sending for inference


def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60 * 60)  # do this once an hour


gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 1000 records to s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/ground_truth_data/2024-02-13-01-45-52/2024/02/13/03/0601.jsonl
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Uploading 1000 records to s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/ground_truth_data/2024-02-13-01-45-52/2024/02/13/04/0602.jsonl
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Uploading 1000 records to s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/ground_truth_data/2024-02-13-01-45-52/2024/02/13/05/0602.jsonl
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from

#### Create a monitoring schedule

In [54]:
##Monitoring schedule name
churn_monitor_schedule_name = (
    f"final-project-xgb-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [55]:
# Create an enpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

# Create the monitoring schedule to execute every hour.
response = churn_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=churn_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: final-project-xgb-monitoring-schedule-2024-02-13-0310


In [56]:
# notice that the status of the schedule below is scheduled
churn_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:075039479415:monitoring-schedule/final-project-xgb-monitoring-schedule-2024-02-13-0310',
 'MonitoringScheduleName': 'final-project-xgb-monitoring-schedule-2024-02-13-0310',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 2, 13, 3, 11, 0, 405000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 2, 13, 3, 11, 6, 141000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-02-13-03-10-59-855',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'final-project-xgboost-v1-wquality-monitor-2024-02-13-0157',
 'ResponseMetadata': {'RequestId': '5e0603a8-8f6e-4efa-92bf-a6f03c9b3034',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5e0603a8-8f6e-4efa-92bf-a6f03c9b3034',
   'content-type': 'application/x-amz-json-1.1

In [57]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to luanch upto 20 min after the hour.
executions = churn_model_quality_monitor.list_executions()
executions

No executions found for schedule. monitoring_schedule_name: final-project-xgb-monitoring-schedule-2024-02-13-0310


[]

In [58]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = churn_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution...................................................................................................................................................................................................................................................................................
Execution found!


In [59]:
while not executions:
    executions = churn_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

..................

{'ProcessingInputs': [{'InputName': 'groundtruth_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/ground_truth_data/2024-02-13-01-45-52/2024/02/13/03',
    'LocalPath': '/opt/ml/processing/groundtruth/2024/02/13/03',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/datacapture/final-project-xgboost-v1-wquality-monitor-2024-02-13-0157/AllTraffic/2024/02/13/03',
    'LocalPath': '/opt/ml/processing/input_data/final-project-xgboost-v1-wquality-monitor-2024-02-13-0157/AllTraffic/2024/02/13/03',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputN

##### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - job exceeded max runtime or was manually stopped.

In [60]:
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = churn_model_quality_monitor.list_executions()[-1]
    execution = churn_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Waiting for execution to finish!
groundtruth-merge-202402130400-db0d7669f505b5fc1b76c060 job status: Completed
groundtruth-merge-202402130400-db0d7669f505b5fc1b76c060 job exit message, if any: None
groundtruth-merge-202402130400-db0d7669f505b5fc1b76c060 job failure reason, if any: None
Execution status is: CompletedWithViolations
{'MonitoringScheduleName': 'final-project-xgb-monitoring-schedule-2024-02-13-0310', 'ScheduledTime': datetime.datetime(2024, 2, 13, 4, 0, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2024, 2, 13, 4, 2, 48, 278000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 2, 13, 4, 18, 30, 777000, tzinfo=tzlocal()), 'MonitoringExecutionStatus': 'CompletedWithViolations', 'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:075039479415:processing-job/model-quality-monitoring-202402130400-db0d7669f505b5fc1b76c060', 'EndpointName': 'final-project-xgboost-v1-wquality-monitor-2024-02-13-0157'}
====STOP==== 
 No completed executions to inspect further. Plea

We can see that we did encounter violations in the monitoring job.

In [61]:
latest_execution = churn_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

Report Uri: s3://sagemaker-us-east-1-075039479415/churn-prediction-xgboost/baselining/results/final-project-xgboost-v1-wquality-monitor-2024-02-13-0157/final-project-xgb-monitoring-schedule-2024-02-13-0310/2024/02/13/04


The report can be found in S3 in the path above. Let's examine the violations.

In [64]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10).iloc[:, ::-1]

,metric_name,description,constraint_check_type
0,auc,Metric auc with 0.5112923189568007 +/- 0.006474879084456501 was LessThanThreshold '0.8881365911098029',LessThanThreshold
1,precision,Metric precision with 0.7350427350427351 +/- 0.0035355949570934327 was LessThanThreshold '0.8',LessThanThreshold
2,truePositiveRate,Metric truePositiveRate with 0.10109717868338558 +/- 0.002418294805042171 was LessThanThreshold '0.37209302325581395',LessThanThreshold
3,f1,Metric f1 with 0.1777471581122976 +/- 0.003686149131763027 was LessThanThreshold '0.5079365079365079',LessThanThreshold
4,accuracy,Metric accuracy with 0.3258966393674103 +/- 0.001473544573075026 was LessThanThreshold '0.845771144278607',LessThanThreshold
5,falsePositiveRate,Metric falsePositiveRate with 0.09403437815975733 +/- 0.004185996257195273 was GreaterThanThreshold '0.02531645569620253',GreaterThanThreshold
6,trueNegativeRate,Metric trueNegativeRate with 0.9059656218402427 +/- 0.0041859962571952425 was LessThanThreshold '0.9746835443037974',LessThanThreshold
7,falseNegativeRate,Metric falseNegativeRate with 0.8989028213166144 +/- 0.002418294805042183 was GreaterThanThreshold '0.627906976744186',GreaterThanThreshold
8,recall,Metric recall with 0.10109717868338558 +/- 0.002418294805042171 was LessThanThreshold '0.37209302325581395',LessThanThreshold
9,f2,Metric f2 with 0.1221706601003883 +/- 0.0028152369842135636 was LessThanThreshold '0.4166666666666667',LessThanThreshold


We can tell that the performance violated several metrics like accuracy, recall, f1 score, etc.

## Analyze model quality CloudWatch metrics <a id='analyze-cloudwatch-metrics'></a> 

In [66]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
]

In [67]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

recall_best_constant_classifier
f2
recall
true_negative_rate
f0_5
false_negative_rate
precision_best_constant_classifier
false_positive_rate
f1
true_positive_rate
au_prc
accuracy
total_number_of_violations
accuracy_best_constant_classifier
f2_best_constant_classifier
precision
f1_best_constant_classifier
f0_5_best_constant_classifier
auc


Let's create an alarm for when the monitoring job sees a dip bellow accepted f1 score

In [68]:
alarm_name = "MODEL_QUALITY_F1_SCORE"
alarm_desc = (
    "Trigger an CloudWatch alarm when the f1 score drifts away from the baseline constraints"
)
mdoel_quality_f1_drift_threshold = (
    0.9 # set high to trigger the alarm easily
)
metric_name = "f1"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_f1_drift_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': '0f4403a1-1339-4b7d-9a7b-ed5f3fdd4145',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0f4403a1-1339-4b7d-9a7b-ed5f3fdd4145',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Tue, 13 Feb 2024 04:58:15 GMT'},
  'RetryAttempts': 0}}

<img src="./pictures/violated cloudwatch alarm.png" style="height:300px" />

To shut down all resources and clean up:

In [69]:
# churn_model_quality_monitor.delete_monitoring_schedule()
# sleep(60)  # actually wait for the deletion

# predictor.delete_model()
# predictor.delete_endpoint()